# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Откроем файл и убедимся, что с данными действительно всё в порядке. Имеем 5 признаков: Количество звонков, Суммарное количество минут, Количество сообщений, Использованный трафик и Тариф.
В массиве данных 3214 строки, пропусков нет.
Признаки Количество звонков и Количество сообщений содержит только целые числа. Можно сменить тип данных на int.

In [3]:
for column in df[['calls','messages']]:
    df[column] = df[column].astype("int")

Также на всякий случай проверим данные на явные дубликаты.

In [4]:
df.duplicated().sum()

0

Явных дубликатов нет. Теперь точно можно приступать к работе.

## Разбейте данные на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки в пропорциях 3:1:1.

In [5]:
features = df.drop(['is_ultra'], axis=1) #Отделим данные с признаками, удалив целевой
target = df['is_ultra'] #Целевой признак вынесем отдельно
#Теперь сначала отделим обучающую выборку, она составит 60% от общего количества.
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)
#От тестовой выборки нам необходимо отделить валидационную в равном процентном соотношении.
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, test_size=0.5, random_state=12345) 
#Проверим правильность размеров полученных выборок:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


Процентное соотношение получилось верным, можем приступать к исследованию.

## Исследуйте модели

Исследуем 3 модели машинного обучения: Decision Tree Classifier, RandomForestClassifier и Logistic Regression.

### Decision Tree Classifier

In [6]:
best_model = None
best_depth = 0
best_result = 0
for depth in range(1, 11):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучаем модель с заданной глубиной дерева
	model.fit(features_train, target_train) # обучаем модель
	predictions_valid = model.predict(features_valid) # получаем предсказания модели
	result = accuracy_score(predictions_valid, target_valid) # посчитаем качество модели
	if result > best_result:
		best_model = model
		best_depth = depth
		best_result = result
        
print('best max_depth =', 
      best_depth, 
      '; ', 
      'best Accuracy = ', 
      best_result, 
      end =''
     ) 

best max_depth = 3 ;  best Accuracy =  0.7853810264385692

Получаем в модели "Решающее дерево" лучший гиперпараметр глубины дерева равный 3, доля правильных ответов составляет 0.785

### RandomForestClassifier

In [12]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0

for est in range(10, 21):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) # обучим модель с заданным количеством деревьев
        model.fit(features_train, target_train) # обучим модель на тренировочной выборке
        result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
        if result > best_result:
            best_model = model # сохраним наилучшую модель
            best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных
            best_est = est
            best_depth = depth
    
print('best n_estimators =', 
      best_est, 
      '; ',
      'best max_depth =', 
      best_depth, 
      '; ', 
      'best Accuracy = ', 
      best_result, 
      end =''
     ) 

best n_estimators = 12 ;  best max_depth = 6 ;  best Accuracy =  0.8040435458786936

В модели "Случайный лес" получим лучшие гиперпараметры: для количества деревьев - 8 и для глубины дерева - 8. Доля правильных ответов составит 0.802 - пока это лучший результат.

### Logistic Regression

In [8]:
model = LogisticRegression(random_state=12345) 
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
score = accuracy_score(predictions_valid, target_valid)
print('Accuracy LogisticRegression:', score, end ='')

Accuracy LogisticRegression: 0.7107309486780715

В модели "Логистическая регрессия" доля правильных ответов составляет 0.711, что ниже чем у предыдущих двух моделей.

## Проверьте модель на тестовой выборке

Лучший результат показала модель "Случайный лес", поэтому на тестовой выборке испытаем именно её.

In [13]:
model = RandomForestClassifier(random_state=12345, n_estimators = 12, max_depth = 6)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result_test = accuracy_score(predictions_test, target_test)
print('Accuracy RandomForestClassifier:', result_test)

Accuracy RandomForestClassifier: 0.7947122861586314


Модель показала высокие результаты и на тестовой выборке, доля правильных ответов изменилась совсем незначительно - 0.796.

## (бонус) Проверьте модели на адекватность

Проверим на адекватность нашу модель с помощью простейшей модели DummyClassifier, которая всегда предсказывает наиболее часто встречающийся класс. Так мы получим контрольные данные для сравнительной оценки построенной нами модели. К тому же мы сможем быть уверены в качестве ее работы, например, в том, что она выдает не просто набор случайно подобранных данных.


[источник](https://habr.com/ru/company/plarium/blog/512332/ "источник")

In [10]:
model_dummy = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dummy.fit(features_train, target_train)
result_dummy = model_dummy.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_dummy)
print('Accuracy RandomForestClassifier:', result_test)

Accuracy DummyClassifier: 0.7060653188180405
Accuracy RandomForestClassifier: 0.7962674961119751


Доля правильных ответов у простейшей модели значительно ниже, значит наша модель адекватна и эффективна.

## Общий вывод

Данные были предоставлены корректные, предобработка как таковая не понадобилась.
Исследовано было три модели: Decision Tree Classifier, RandomForestClassifier и Logistic Regression.
Самый низкий результат показала модель Logistic Regression с результатом 0.711.
Лучший результат показала модель RandomForestClassifier с гиперпараметрами n_estimators равным - 8 и max_depth равным 8. Доля правильных ответов составила 0.802. На тестовой выборке результат составил 0.796. 
При проверке на адекватность использовалась модель DummyClassifier, её результат составил 0.706, что доказало адекватность модели RandomForestClassifier.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
